In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
credits = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_credits.csv")
movies = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_movies.csv")

In [ ]:
credits.shape,movies.shape

In [ ]:
credits

In [ ]:
movies

In [ ]:
credits = credits.rename(columns={"movie_id":"id"})
credits

In [ ]:
data = movies.merge(credits,on="id")
data

In [ ]:
data = data.drop(["homepage","title_x","title_y","status","production_countries"],axis=1)
data

In [ ]:
data.info()

In [ ]:
R = data.vote_average
v = data.vote_count
m = data.vote_count.quantile(0.70)
C = data.vote_average.mean()

In [ ]:
data['weighted_average']=((R*v)+ (C*m))/(v+m)

In [ ]:
data

In [ ]:
data_sort=data.sort_values('weighted_average',ascending=False)

In [ ]:
data_sort[['original_title', 'vote_count', 'vote_average', 'weighted_average', 'popularity']].head(20)

In [ ]:
weight_average=data_sort.sort_values('weighted_average',ascending=False)
plt.figure(figsize=(12,6))
axis1=sns.barplot(x=weight_average['weighted_average'].head(10), y=weight_average['original_title'].head(10), data=weight_average)

In [ ]:
popularity = data_sort.sort_values('popularity',ascending=False)
plt.figure(figsize=(12,6))
sns.barplot(y=popularity.original_title.head(10),x=popularity.popularity.head(10))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaling=MinMaxScaler()
movie_scaled_df=scaling.fit_transform(data[['weighted_average','popularity']])
movie_normalized_df=pd.DataFrame(movie_scaled_df,columns=['weighted_average','popularity'])
movie_normalized_df.head()

In [ ]:
data[['normalized_weight_average','normalized_popularity']]= movie_normalized_df

In [ ]:
data

In [ ]:
data['score'] = data['normalized_weight_average'] * 0.5 + data['normalized_popularity'] * 0.5
data

In [ ]:
movies_scored_df = data.sort_values(['score'], ascending=False)

In [ ]:
movies_scored_df[['original_title', 'normalized_weight_average', 'normalized_popularity', 'score']].head(20)

In [ ]:
scored_df = data.sort_values('score', ascending=False)

plt.figure(figsize=(16,6))

ax = sns.barplot(x=scored_df['score'].head(10), y=scored_df['original_title'].head(10), data=scored_df, palette='deep')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')
data['overview'] = data['overview'].fillna('')

In [ ]:
data

In [ ]:
matrix = tfv.fit_transform(data['overview'])

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(matrix,matrix)

In [ ]:
indices = pd.Series(data.index, index = data['original_title']).drop_duplicates()
indices

In [ ]:
def give_rec(title, sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[1:11]
    movie_indices = [i[0] for i in sig_scores]
    return data['original_title'].iloc[movie_indices]

In [ ]:
give_rec('Finding Nemo')